In [13]:
import numpy as np
import matplotlib.pyplot as plt
import os
# import tensorrt
#ImportError: Python version mismatch: module was compiled for Python 3.10, but the interpreter version is incompatible: 3.9.17 (main, Jul  5 2023, 20:41:20) 
#[GCC 11.2.0].
# print(tf.__file__)
# import os
# print(os.environ['LD_LIBRARY_PATH'])
from tensorflow.keras.layers import Input, Dense,Conv1D, MaxPooling1D, UpSampling1D, Flatten, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras import backend as K
from tensorflow.python.ops import math_ops
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tqdm import tqdm
from scipy.io import wavfile
import random
import itertools
import json
# from google.colab import drive

In [14]:
pip install import-ipynb

/bin/bash: /home/martin/.local/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.


In [51]:
def cut(arr, length):
  idx = len(arr)%length
  out = []
  while(idx+length <= len(arr)):
    out.append(arr[idx:idx+length])
    idx += length
  return np.array(out)

def loadSong(fName, numTotalSongs = 1):
  fs, data = wavfile.read(inpathTrain + fName)
  all_data = [data]

  if numTotalSongs > 1:
    seed = sum([ord(char) for char in fName])
    random.seed(seed)
    file_nams = random.sample(fileNames[:170], numTotalSongs-1)
    for name in file_nams:
      fs, data1 = wavfile.read(inpathTrain + name)
      all_data.append(data1)

  
  concatenated_data = np.concatenate(all_data)
  if concatenated_data.ndim > 1:
    mono_data = np.mean(concatenated_data, axis=1)
  else:
    mono_data = concatenated_data

  # # all_data.append(data)

  # # all_data = np.concatenate(data, data1)
  # for name in file_nams:
  #   fs, data1 = wavfile.read(inpathTrain + name)
  #   data = np.concatenate(data, data1)
  #   # all_data.append(data)

  # all_data = np.array(data)
  # if data.ndim > 1:
  #   mono_data = np.mean(all_data, axis=1)
  # else:
  #   mono_data = all_data

  return mono_data.astype('int16')

def loadSongCut(fName, silence_prob = 0, numTotalSongs = 1, percentage_of_song = 1):
  data = loadSong(fName, numTotalSongs)
  data = cut(data, snippitLength)

  # Replace rows with silence based on silence_prob
  if silence_prob!=0:
    num_rows = data.shape[0]
    num_silence_rows = int(num_rows * silence_prob)
    silence_rows = np.zeros((num_silence_rows, data.shape[1]))
    data[:num_silence_rows, :] = silence_rows


  scaler[fName] = MinMaxScaler()
  #data = quadratic_scaler(data, 5)
  data = scaler[fName].fit_transform(data)

  Xt, Xv = train_test_split(data, test_size=0.3, random_state=42)
  if percentage_of_song != 1:
    index_t = int(len(Xt)*percentage_of_song)
    index_v = int(len(Xv)*percentage_of_song)
    Xt = Xt[:index_t]
    Xv = Xv[:index_v]
  return np.array(Xt), np.array(Xv)




def loadSongCut_tensor(fName, silence_prob = 0):

  Xt, Xv = loadSongCut(fName, silence_prob)

  # Convert NumPy arrays to TensorFlow tensors
  # Xt_tensor = tf.constant(Xt, dtype=tf.float32)
  # Xv_tensor = tf.constant(Xv, dtype=tf.float32)
  Xt_tensor = tf.constant(Xt, dtype=tf.int16)
  Xv_tensor = tf.constant(Xv, dtype=tf.int16)

  # Create datasets from TensorFlow tensors
  dataset_Xt = tf.data.Dataset.from_tensor_slices(Xt_tensor)
  dataset_Xv = tf.data.Dataset.from_tensor_slices(Xv_tensor)

  batch_size = 32  # Adjust according to memory availability and training efficiency
  buffer_size = 10000  # Set to a value larger than the dataset size but still fits in memory
  prefetch_size = 1  # Increase if input pipeline is a bottleneck, otherwise keep it low


  # Configure the datasets
  dataset_Xt = dataset_Xt.batch(batch_size)  # Set batch size
  dataset_Xt = dataset_Xt.shuffle(buffer_size=buffer_size)  # Enable shuffling
  dataset_Xt = dataset_Xt.prefetch(prefetch_size)  # Enable prefetching

  dataset_Xv = dataset_Xv.batch(batch_size)  # Set batch size
  dataset_Xv = dataset_Xv.shuffle(buffer_size=buffer_size)  # Enable shuffling
  dataset_Xv = dataset_Xv.prefetch(prefetch_size)  # Enable prefetching

  return dataset_Xt, dataset_Xv

def digital_decibel(x):
  if (x>0):
    decibels = 1/ (10 * np.log10(x/1))
  else:
    decibels = 1 / (-1*  10 * np.log10(-x/1))
  return decibels

def quadratic_scaler(x, n):
  v = [i**n for i in x]
  return v

# s = loadSong('2633_ps10_03.wav', 2)
# wavfile.write('test.wav', 44100, s)

# t, v = loadSongCut('2633_ps10_03.wav', numTotalSongs = 2, percentage_of_song = 0.1)
t, v = loadSongCut('2633_ps10_03.wav') # ((93324, 64), (39996, 64))
# t, v = loadSongCut('2633_ps10_03.wav', numTotalSongs = 3, percentage_of_song = 0.3)
t.shape, v.shape
# (45307, 64)

((93324, 64), (39996, 64))

In [16]:
def snipLoss(y_true, y_pred):
  snipWeight = tf.convert_to_tensor([int(np.cosh(x)) for x in range(-5, 5, snippitLength)], dtype='float32')

  loss = math_ops.squared_difference(y_true,y_pred)
  loss = math_ops.Mul(x = loss,y = snipWeight)
  loss = math_ops.log1p(loss)
  return loss

def si_snr2(y_true, y_pred):
  # Remove extra dimensions
  y_true = tf.squeeze(y_true)
  y_pred = tf.squeeze(y_pred)

  # Compute the scaling factor
  scale = tf.reduce_sum(y_true * y_pred) / tf.reduce_sum(y_pred * y_pred)

  # Compute the estimated source and the target source
  est_source = scale * y_pred
  target_source = y_true

  # Compute the noise source
  noise_source = est_source - target_source

  # Compute the SI-SNR
  numerator = tf.reduce_sum(target_source * est_source, axis=-1)
  denominator = tf.reduce_sum(noise_source * noise_source, axis=-1)
  si_snr = 10 * tf.math.log(numerator / denominator + 1e-8) / tf.math.log(10.0)

  # Return the average SI-SNR
  return tf.reduce_mean(si_snr)

def si_snr(target, estimate):
  # target and estimate are tensors of shape (batch_size, time_steps)
  # compute the dot product of target and estimate along the time axis
  dot = tf.reduce_sum(target * estimate, axis=-1, keepdims=True)
  # compute the energy of target along the time axis
  energy = tf.reduce_sum(target ** 2, axis=-1, keepdims=True)
  # compute the scaled target
  scaled_target = dot * target / energy
  # compute the noise
  noise = estimate - scaled_target
  # compute the SI-SNR in decibels
  si_snr = 10 * tf.math.log(tf.reduce_sum(scaled_target ** 2, axis=-1) / tf.reduce_sum(noise ** 2, axis=-1)) / tf.math.log(10.0)
  # return the SI-SNR tensor of shape (batch_size,)
  return si_snr

In [17]:
# paths
# drive.mount('/content/drive')
# inpathTrain = "/content/drive/MyDrive/Machine Learning/Autoencoder/train_data/"
# inpathOut = "/content/drive/MyDrive/Machine Learning/Autoencoder/output/"
inpathTrain = "/mnt/e/data/SynologyDrive/Uni/mSem02/Machine Learning/Project_Audio_Autoencoder/musicnet_midis/BOT/Mixdown/output/"
inpathOut =   "/home/martin/martin_user_data/jupyter_notebooks/autoencoder_ml/output/"
output_folder = "output/Versuch3_13.07.2023/"
fileNames = os.listdir(inpathTrain)
random.seed(42)
fileNames = random.sample(fileNames, len(fileNames))

hyperparamsearch_folder = output_folder + 'hyperparamsearch'
train_compression_rates_folder = output_folder + 'train_compression_rates'

if not os.path.exists(hyperparamsearch_folder):
    os.mkdir(hyperparamsearch_folder)

if not os.path.exists(train_compression_rates_folder):
    os.mkdir(train_compression_rates_folder)

scaler = {}

# global variables
samplerate = 44_100
snippitLength = 64

loss_fct = snipLoss




In [18]:
####################################
#####  plot history

def plot_loss(ax, network_history):
    loss = np.concatenate([network_history[key].history['loss'] for key in network_history.keys()])
    val_loss = np.concatenate([network_history[key].history['val_loss'] for key in network_history.keys()])

    ax.set_xlabel('Epochs')
    ax.set_ylabel('Loss')
    ax.set_title('Loss')
    ax.plot(loss, label='Training')
    ax.plot(val_loss, label='Validation')
    ax.legend()

def plot_si_snr(ax, network_history):
    si_snr = np.concatenate([network_history[key].history['si_snr'] for key in network_history.keys()])
    val_si_snr = np.concatenate([network_history[key].history['val_si_snr'] for key in network_history.keys()])

    ax.set_xlabel('Epochs')
    ax.set_ylabel('SI_SNR')
    ax.set_title('SI-SNR')
    ax.plot(si_snr, label='Training')
    ax.plot(val_si_snr, label='Validation')
    ax.legend()

def plot_history(network_history, name):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6), sharex=True, sharey=False)

    plot_loss(ax[0], network_history)
    plot_si_snr(ax[1], network_history)

    plt.tight_layout()
    plt.savefig(name)
    # plt.show()
    plt.clf()


In [19]:
####################################
#####  buildModel (Hyperparameter grid search)
# tf.random.set_seed(42)
def lr_schedule(epoch):
  if epoch < 10:
      return 0.01
  else:
      return 0.001

# def buildModel(compression_ratio = 0.5, numDense = 2, numConv = 8,numConvLayer = 2, loss_fct = snipLoss, use_bias = False):
def buildModel(compression_ratio = 0.5, numDense = 1, numConv = 8,numConvLayer = 0, loss_fct = snipLoss, use_bias = False, learning_rate = 0.001):



  latentSize = int(compression_ratio*snippitLength)

  # keep tensorflow from allocating more memory as it currently needs
  physical_devices = tf.config.experimental.list_physical_devices('GPU')
  for i in physical_devices:
      tf.config.experimental.set_memory_growth(i, True)
  tf.device('/device:GPU:0')

  input = Input(shape=(snippitLength,1))
  x = input

  # Convolutional part of encoder
  for i in range(numConvLayer):
    x = Conv1D(numConv, 5, activation='relu', padding='same')(x)
    x = MaxPooling1D(2, padding = 'same')(x)

  convShape = x.shape
  # calculate flatten dimension
  flsize = 1
  for i in x.shape:
    if(i != None):
      flsize*= i

  # print(f'{latentSize} {flsize}')
  x = Flatten()(x)


  # Dense part of encoder
  denses = [int(i) for i in np.linspace(flsize, latentSize, numDense+1)]
  for i in denses[1:]:
    x = Dense(i, activation='relu', use_bias=use_bias)(x)

  encoded = x

  # for i in denses:

  # Dense part of decoder
  x = encoded
  for i in denses[::-1][1:]:
    if(numConvLayer == 0 and i == snippitLength):
      x = Dense(i, activation='sigmoid')(x)
    else:
      x = Dense(i, activation='relu', use_bias=use_bias)(x)

  if(numConvLayer == 0):
    decoded = x

  x = Reshape(convShape[1:])(x)

  # Convolutional part of decoder
  for i in range(numConvLayer):
    x = Conv1D(numConv,5, activation='relu', padding='same')(x)
    x = UpSampling1D(2)(x)
  if(numConvLayer != 0):
    decoded = Conv1D(1,5, activation='sigmoid', padding='same')(x)

  autoencoder = Model(input, decoded)
  autoencoder = Model(input, Flatten()(decoded))

  
  autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss=loss_fct, metrics=[si_snr])
  
  print(f'current model: ratio={compression_ratio},numDense={numDense},numConv={numConv},numConvLayer={numConvLayer}')
  # autoencoder.summary()
  return autoencoder



get_custom_objects()['snipLoss'] = snipLoss
get_custom_objects()['si_snr'] = si_snr
Xt, Xv = loadSongCut('1727_schubert_op114_2.wav')
buildModel().fit(Xt[:2], Xt[:2],
            epochs=1,
            batch_size=512,
            shuffle=True,
            validation_data=(Xv[:2], Xv[:2]))

2023-07-16 12:34:07.670780: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-16 12:34:07.818916: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-16 12:34:07.818951: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-16 12:34:07.820401: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-07-16 12:34:07.820431: I tensorflow/compile

current model: ratio=0.5,numDense=1,numConv=8,numConvLayer=0


2023-07-16 12:34:10.557900: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7fe890061d60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-16 12:34:10.557949: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-07-16 12:34:10.563604: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-07-16 12:34:11.668467: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8902
2023-07-16 12:34:11.694874: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:530] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/

1/1 [==============================] - 3s 3s/step - loss: 0.5445 - si_snr: 12.8107 - val_loss: 0.8248 - val_si_snr: 13.4159


In [20]:
param_space = {'compression_ratio' : [0.1, 0.2, 0.3, 0.4],
               'numDense' : [3, 4, 5, 6],
               'numConv' : [8, 16],
               'numConvLayer' : [0, 1, 2]}



# param_space = {'compression_ratio' : np.linspace(0.1,0.9,2),
#                'numDense' : [2, 3],
#                'numConv' : [8, 6],
#                'numConvLayer' : np.linspace(0.1,0.2,2)} # small 1
# param_space = {'compression_ratio' : [0.1, 0.2],
#                'numDense' : [3,4],
#                'numConv' : [4, 8],
#                'numConvLayer' : [0, 1]} # small 2


param_space = {'compression_ratio' : [0.2],
               'numDense' : [2, 3, 4, 5, 6],
               'numConv' : [8, 16, 24, 32],
               'numConvLayer' : [0, 1, 2, 3, 4]}

param_space = {'compression_ratio' : [0.2],
               'numDense' : [2, 3, 4],
               'numConv' : [32, 64],  # 128 too much memory need
               'numConvLayer' : [1, 2]}

param_space = {'compression_ratio' : [0.2],
               'numDense' : [1],
               'numConv' : [32, 64],  # 128 too much memory need
               'numConvLayer' : [1, 2]}


param_space = {'compression_ratio' : [0.2],
               'numDense' : [1],
               'numConv' : [64, 96, 128],
               'numConvLayer' : [4, 6]}


# param_space = {'compression_ratio' : [0.2],
#                'numDense' : [5, 6],
#                'numConv' : [24, 32],
#                'numConvLayer' : [ 3, 4]}

numHyperTrainSongs = 120
numHyperTrainSongs = 170
numHyperTrainSongs = 70
numHyperEpochs = 1
# numHyperTrainSongs = 170
# numHyperEpochs = 10

value_combis = itertools.product(*[v for v in param_space.values()])
param_combis = []
for combi in value_combis:
  param_combi = {key: value for key, value in zip(param_space.keys(), combi)}
  if param_combi['numConvLayer'] == 0:
    param_combi['numConv'] = 0
  param_combis.append(param_combi)

# param_combis = [
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 32, 'numConvLayer': 2}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 32, 'numConvLayer': 3}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 2}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 64, 'numConvLayer': 3}, #47.8
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 2}, # 48.5
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 3}, # 48.5
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 224, 'numConvLayer': 2}, # 49.6
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 224, 'numConvLayer': 3}] # 49.6

#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 2}, # 48.2
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 3},
# param_combis = [
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 4},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 5},
#  {'compression_ratio': 0.2, 'numDense': 1, 'numConv': 96, 'numConvLayer': 6}]

time_per_combi = 2.6
time_per_combi = 3*3.5*4
print(f'estimated time = {time_per_combi*len(param_combis)/60:.1f} h ({len(param_combis)} sets)')
# param_combis = param_combis[-1:]
# param_combis = reversed(param_combis)
param_combis = param_combis[5:]
param_combis

estimated time = 4.2 h (6 sets)


[{'compression_ratio': 0.2, 'numDense': 1, 'numConv': 128, 'numConvLayer': 6}]

In [21]:
# ####################################
# #####  Hyperparameter grid search
# import stopwatch as sw

# t = sw.stopwatch(title='gridsearch', time_unit='s')

# # Load existing results from the JSON file if it exists
# existing_results = []
# existing_file_path = output_folder + 'hyperparamsearch/' + 'searchResults_170songs_3epochs.json'
# existing_file_path = output_folder + 'hyperparamsearch/' + 'searchResults_170songs_1epochs_final.json'
# if os.path.exists(existing_file_path):
#     with open(existing_file_path, 'r') as file:
#         existing_results = json.load(file)

# random.seed(42)
# filename_random = random.sample(fileNames, 170)
# search_results = []
# model_save_path = output_folder + 'hyperparamsearch/' + f'model.keras'
# if os.path.exists(model_save_path):
#     os.remove(model_save_path)

# for hyperParamSet in tqdm(param_combis):
#   autoencoder = buildModel(hyperParamSet['compression_ratio'],
#                            hyperParamSet['numDense'],
#                            hyperParamSet['numConv'],
#                            hyperParamSet['numConvLayer'])

#   # autoencoder = buildModel()
#   histories = {}
#   # numHyperTrainSongs = 10
#   batch_size=4096
#   t.task('hyperparam')
#   for idx, filename_train in tqdm(enumerate(filename_random[:numHyperTrainSongs])):
#     Xt, Xv = loadSongCut(filename_train)
#     histories[filename_train] = autoencoder.fit(Xt, Xt,
#                 epochs=numHyperEpochs,
#                 batch_size=batch_size,
#                 shuffle=True,
#                 validation_data=(Xv, Xv))
#     del Xt
#     del Xv
#     if (idx % int(5/numHyperEpochs) == 0) and (idx != 0):
#       autoencoder.save(model_save_path)
#       del autoencoder
#       autoencoder = tf.keras.models.load_model(model_save_path)
#   t.stop()
#   del autoencoder
#   tf.keras.backend.clear_session()

#   pdfname = f'HyperParOpt, compression_ratio= {hyperParamSet["compression_ratio"]:.1f}, numDense= {hyperParamSet["numDense"]}, numConvLayer= {hyperParamSet["numConvLayer"]}, numConv= {hyperParamSet["numConv"]}.pdf'
#   plot_history(histories, output_folder + 'hyperparamsearch/' + pdfname)

#   loss = []
#   val_loss = []
#   train_si_snr = []
#   val_si_snr = []
#   for key in histories.keys():
#     loss.append(histories[key].history['loss'])
#     val_loss.append(histories[key].history['val_loss'])
#     train_si_snr.append(histories[key].history['si_snr'])
#     val_si_snr.append(histories[key].history['val_si_snr'])
#   loss         = np.concatenate(loss)
#   val_loss     = np.concatenate(val_loss)
#   train_si_snr = np.concatenate(train_si_snr)
#   val_si_snr   = np.concatenate(val_si_snr)

#   best_val_epoch    = np.argmax(val_si_snr)
#   best_val_si_snr   = np.max(val_si_snr)
#   best_val_loss     = np.min(val_loss)
#   best_train_si_snr = np.max(train_si_snr)
#   best_train_loss   = np.min(loss)

#   search_results.append({
#     **hyperParamSet,
#     'best_val_epoch': best_val_epoch,
#     'best_val_si_snr': best_val_si_snr,
#     'best_val_loss': best_val_loss,
#     'best_train_si_snr': best_train_si_snr,
#     'best_train_loss': best_train_loss
#   })


#   latest_results = [{k: int(v) if isinstance(v, np.int64) else v for k, v in d.items()} for d in search_results]

#   # Merge existing results and latest results
#   all_results = existing_results + latest_results

#   # Write all results to the JSON file
#   with open(existing_file_path, 'w') as file:
#       json.dump(all_results, file, indent='')


In [22]:
# # Merge existing results and latest results
# all_results = existing_results + latest_results

# # Write all results to the JSON file
# with open(existing_file_path, 'w') as file:
#     json.dump(all_results, file, indent='')

In [23]:
# 

# results = [{k: int(v) if isinstance(v, np.int64) else v for k, v in d.items()} for d in search_results]

# with open(output_folder + 'hyperparamsearch/' + 'searchResults.json', 'w') as file:
#     json.dump(results, file, indent = '')

In [24]:
# import import_ipynb
# from plot_hyperparameter import *
# hyperparameter_Plot(results, output_folder)

In [25]:
####################################
#####  evaluate test songs

# numTestSongs = 2
def evaluateTestSongs(autoencoder, num = 0):
  index = 0
  test_evaluated = []
  if num!=0:
    numTestSongs = num
  print(f'evaluating {numTestSongs} test songs')
  for songname in tqdm(reversed(fileNames[-numTestSongs:])):
      # songname = '1727_schubert_op114_2.wav'
      song = loadSong(songname)
      # wavfile.write(inpathOut + songname + '.wav', samplerate, song)
      songSnip = cut(song, snippitLength)

      songSnip_transformed = MinMaxScaler().fit_transform(songSnip)
      test_loss, test_si_snr = autoencoder.evaluate(songSnip_transformed, songSnip_transformed, verbose=2)

      test_evaluated.append([songname, test_loss, test_si_snr])
  return test_evaluated

In [26]:
####################################
#####  predict unknown song

def plotWave(autoencoder, name, compression_ratio):
  exampleSong = fileNames[-1]
  # exampleSong = '1727_schubert_op114_2.wav'
  orig = loadSong(exampleSong)
  origSnip = cut(orig, snippitLength)
  orig = np.concatenate(origSnip)

  if(exampleSong in scaler.keys()):
    scaler_Example = scaler[exampleSong]
    origSnip_transformed = scaler_Example.transform(origSnip)
  else:
    scaler_Example = MinMaxScaler()
    origSnip_transformed = scaler_Example.fit_transform(origSnip)

  # autoencode song
  a = autoencoder.predict(origSnip_transformed)
  a = a.reshape(-1, snippitLength)
  XpredSnip = scaler_Example.inverse_transform(a)

  silence = np.zeros((1,snippitLength), dtype = 'int16')
  a = scaler_Example.transform(silence)
  a = autoencoder.predict(a)
  a = a.reshape(-1, snippitLength)
  Xsilence = scaler_Example.inverse_transform(a)[0]

  # remove noise generated by silence
  XpredSnip_minussilence = [i-Xsilence for i in XpredSnip]
  Xpred = np.concatenate(XpredSnip_minussilence).astype('int16')

  test_loss, test_si_snr = autoencoder.evaluate(origSnip_transformed, origSnip_transformed)

  output_wav_name = f'snln={snippitLength}_cmpr={compression_ratio:.1f}_loss={loss_fct.__name__}_songs={numTopoTrainSongs}_SNR={test_si_snr:.1f}.wav'
  wavfile.write(output_folder + output_wav_name, samplerate, Xpred)
  print(f"file saved: {output_wav_name}")

  plt.plot(orig, linewidth = 0.1)
  plt.plot(orig-Xpred, linewidth = 0.1)
  plt.savefig(name + "whole.pdf")
  plt.clf()
  ####################################
  #####  see difference in waveform detailed
  nrows = 2
  ncols = 6
  snips = [0, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000]

  fig, ax = plt.subplots(nrows, ncols, figsize=(6*ncols, 6*nrows), sharey = True, sharex = True)
  s = 0
  for i in range(nrows):
    for j in range(ncols):
      ax[i][j].plot(origSnip[snips[s]], linewidth = 0.5, c = 'b')
      ax[i][j].plot(XpredSnip_minussilence[snips[s]], linewidth = 0.5, c = 'r')
      s +=1
  plt.savefig(name + "snip_corrected.pdf")
  plt.clf()

  fig, ax = plt.subplots(nrows, ncols, figsize=(6*ncols, 6*nrows), sharey = True, sharex = True)
  s = 0
  for i in range(nrows):
    for j in range(ncols):
      ax[i][j].plot(origSnip[snips[s]], linewidth = 0.5, c = 'b')
      ax[i][j].plot(XpredSnip[snips[s]], linewidth = 0.5, c = 'r')
      # ax[i][j].plot(XpredSnip_minussilence[snips[s]], linewidth = 0.5, c = 'r')
      s +=1
  plt.savefig(name + "snip_notcorrected.pdf")
  plt.clf()


In [62]:
# Find best topology across different compression ratios and build/train 10 models across compression ratios with that topology
import csv

def read_histories_from_csv(file_path):
    histories = {}
    with open(file_path, 'r') as csvfile:
        reader = csv.reader(csvfile)
        next(reader)  # Skip the header row
        for row in reader:
            filename = row[0]
            loss = float(row[1])
            val_loss = float(row[2])
            histories[filename] = {'loss': [loss], 'val_loss': [val_loss]}
    return histories

# tf.random.set_seed(42+300)

parSet_sum = {}
search_results_json = output_folder + 'hyperparamsearch/' + 'searchResults.json'
search_results_json = 'output/Versuch1_11.07.2023/searchResults.json'
search_results_json = 'output/Versuch3_13.07.2023/hyperparamsearch/searchResults_170songs_5epochs_final.json'
with open(search_results_json, 'r') as file:
    search_results = json.load(file)

for item in search_results:
    numDense = item['numDense']
    numConv = item['numConv']
    numConvLayer = item['numConvLayer']
    best_val_si_snr = item['best_val_si_snr']

    key = (numDense, numConv, numConvLayer)
    if key in parSet_sum:
       parSet_sum[key] += best_val_si_snr
    else:
        parSet_sum[key] = best_val_si_snr

keys = [k for k in parSet_sum.keys()]
si_snr_sum = [parSet_sum[k] for k in keys]
bestParSet = keys[np.argmax(si_snr_sum)]
# print(f'best set : {bestParSet}')

# compression_rates = np.linspace(0.1,0.9,9) # todo
compression_rates = [0.2]
# silence_prob = 0.01

numTopoTrainSongs = 1 # 6 min 
numTopoEpochs = 1

numTotalSongs = 17
percentage_of_song = float(1/(numTotalSongs))

total_num_songs = len(fileNames)
numTopoTrainSongs = int(((total_num_songs*0.7)+1))
# numTopoTrainSongs = numTopoTrainSongs*4
numTopoEpochs = 1

learning_rate = 0.1 # nooo
learning_rate = 0.01 # nooo
learning_rate = 0.0001 #no
learning_rate = 0.0005
learning_rate = 0.005  # 20
learning_rate = 0.003  # 36
learning_rate = 0.0004  # better and slow up trend should be ok
learning_rate = 0.0008  # better and slow up trend
learning_rate = 0.0004 #no
learning_rate = 0.001 #30 ish
learning_rate = 0.0006  # better and slow up trend
learning_rate = 0.0002 #no
learning_rate = 0.00008 #no

numTestSongs = int(total_num_songs*0.3)
model_save_path = output_folder + 'hyperparamsearch/' + f'model_train1_1024_5.keras' #20
model_save_path = output_folder + 'hyperparamsearch/' + f'model_train1_256_3.keras' #20
model_save_path = output_folder + 'train_compression_rates/' + f'model_train1_128_3.keras' #20
model_save_path = output_folder + 'train_compression_rates/' + f'model_train_1_96_3.keras' #100
histories_save_path = output_folder + 'train_compression_rates/' + 'histories_train1_128_3.csv'
histories_save_path = output_folder + 'train_compression_rates/' + 'histories_train_1_96_3.csv'
def lr_schedule(epoch):
  return learning_rate

lr_scheduler = LearningRateScheduler(lr_schedule)
for c in compression_rates:
  # autoencoder = buildModel(c, bestParSet[0],bestParSet[1],bestParSet[2], learning_rate = learning_rate)
  # autoencoder = buildModel(c, 1, 96, 3, learning_rate = learning_rate)
  autoencoder = tf.keras.models.load_model(model_save_path)

  histories = {}
  batch_size=512
  batch_size=2**12
  batch_size=2**8  #256
  batch_size=2**7  #128 # train and val diverging by +0.1
  batch_size=2**6  #64
  for idx, filename_train in tqdm(enumerate(fileNames[120:numTopoTrainSongs:])):
    Xt, Xv = loadSongCut(filename_train, numTotalSongs = numTotalSongs, percentage_of_song = percentage_of_song)
    histories[filename_train] = autoencoder.fit(Xt, Xt,
                epochs=numTopoEpochs,
                batch_size=batch_size,
                shuffle=True,
                validation_data=(Xv, Xv),
                callbacks=[lr_scheduler])
    del Xt
    del Xv
    save_period = int(9/numHyperEpochs)
    save_period = 1
    if (idx % save_period == 0) and (idx != 0):
      autoencoder.save(model_save_path)
      del autoencoder
      autoencoder = tf.keras.models.load_model(model_save_path)

      with open(histories_save_path, 'w', newline='') as csvfile:
          writer = csv.writer(csvfile)
          writer.writerow(['filename', 'loss', 'val_loss'])
          for key, value in histories.items():
              writer.writerow([key, value.history['loss'][0], value.history['val_loss'][0]])
  tf.keras.backend.clear_session()
  
  pdfname = f'BestSet, compression_ratio ={c:.1f}.pdf'
  histories = read_histories_from_csv(histories_save_path)
  plot_history(histories, output_folder + 'train_compression_rates/' + pdfname)
  plotWave(autoencoder, f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}wave_', c)

  autoencoder.save(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}.keras')
  testPerformance = evaluateTestSongs(autoencoder, num = 1)

  with open(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}Performance.csv', 'w', newline='') as file:
    # Create a CSV writer
    writer = csv.writer(file)

    # Write the data row by row
    for row in testPerformance:
        writer.writerow(row)

  break


0it [00:00, ?it/s]

2590/2590 [==============================] - 32s 12ms/step - loss: 0.0022 - si_snr: 45.9880 - val_loss: 0.0019 - val_si_snr: 46.9461 - lr: 8.0000e-05


1it [00:47, 47.41s/it]

3031/3031 [==============================] - 38s 12ms/step - loss: 0.0023 - si_snr: 45.7534 - val_loss: 0.0021 - val_si_snr: 45.9120 - lr: 8.0000e-05


2it [01:31, 45.41s/it]

...

119it [1:23:08, 39.48s/it]

2746/2746 [==============================] - 36s 12ms/step - loss: 0.0022 - si_snr: 46.3513 - val_loss: 0.0021 - val_si_snr: 46.0330 - lr: 8.0000e-05


120it [1:23:50, 40.31s/it]

: 

: 

In [48]:
numTotalSongs = 20
percentage_of_song = 1/(numTotalSongs)
percentage_of_song

0.05

In [28]:
autoencoder = tf.keras.models.load_model(model_save_path)

plotWave(autoencoder, f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}wave_', c)

4890/4890 [==============================] - 20s 4ms/step - loss: 0.0361 - si_snr: 27.9298
file saved: snln=64_cmpr=0.2_loss=snipLoss_songs=68_SNR=27.9.wav


<Figure size 640x480 with 0 Axes>

<Figure size 3600x1200 with 0 Axes>

<Figure size 3600x1200 with 0 Axes>

In [ ]:
  for filename_train in tqdm(fileNames[:numTopoTrainSongs]):
    Xt, Xv = loadSongCut(filename_train, silence_prob = silence_prob)
    histories[filename_train] = autoencoder.fit(Xt, Xt,
                epochs=numTopoEpochs,
                batch_size=2**10,
                shuffle=True,
                validation_data=(Xv, Xv))
    
  pdfname = f'BestSet, compression_ratio ={c:.1f}.pdf'
  plot_history(histories, output_folder + 'train_compression_rates/' + pdfname)

  autoencoder.save(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}.keras')
  testPerformance = evaluateTestSongs(autoencoder)

  with open(f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}Performance.csv', 'w', newline='') as file:
    # Create a CSV writer
    writer = csv.writer(file)

    # Write the data row by row
    for row in testPerformance:
        writer.writerow(row)
  plotWave(autoencoder, f'{output_folder}train_compression_rates/modelCompressionRate:{c:.1f}wave_', c)

  break


In [ ]:
# #numTrainSongs =
# batch_size = 2**10
# epochs = 5
# histories = {}
# for fn in tqdm(fileNames[:numTrainSongs]):
#   Xt, Xv = loadSongCut(fn)
#   Xt = np.array(Xt)
#   Xv = np.array(Xv)
#   histories[fn] = autoencoder.fit(Xt, Xt,
#                 epochs=epochs,
#                 batch_size=batch_size,
#                 shuffle=True,
#                 validation_data=(Xv, Xv))